In [1]:
library(dplyr)
library(stringr)
library(readr)
library(tidyr)
library(lfe)
library(aod)
source('~/covid_mobility/paper_code/utils/adjacency_and_cluster_robust_inference.R')
load('/nfs/sloanlab004/projects/covid_mobility_proj/data/PROCESSED_DATA/county_mobility_dvs.Rdata')


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: Matrix

Attaching package: ‘Matrix’

The following object is masked from ‘package:tidyr’:

    expand


Attaching package: ‘reshape2’

The following object is masked from ‘package:tidyr’:

    smiths

Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


Attaching package: ‘lmtest’

The following object is masked from ‘package:lfe’:

    waldtest



In [2]:
### Loading Data

fp <- '/nfs/sloanlab004/projects/covid_mobility_proj/data/'
safegraph <- read_csv(str_c(fp, 'PROCESSED_DATA/safegraph_social_distancing_aggregate_county.csv'))
counties_long <- read_csv(str_c(fp, 'PROCESSED_DATA/county_policy_long.csv'))
countyInfo <- read_csv(str_c(fp, 'PROCESSED_DATA/countyInfo.csv'))
weather <- read_csv(str_c(fp, 'PROCESSED_DATA/county_weather.csv'))
voteShare <- read_csv(str_c(fp, 'PROCESSED_DATA/county_vote_shares_2016.csv'))
sci <- read_delim(str_c(fp, 'fb_social_connectedness/sci_county/county_county_data.tsv'), delim ='\t', col_types = 'ddd')
WFH = read_csv(str_c(fp, "naics_codes/NAICS_workfromhome.csv")) 
census = read_csv(str_c(fp, 'census_data/cc-est2018-alldata.csv'))
naics = read_csv(str_c(fp, 'PROCESSED_DATA/safegraph_daily_patterns_2_digit_naics_aggregate_county.csv'))
naics_jan = read_csv(str_c(fp, 'PROCESSED_DATA/historic_safegraph/safegraph_daily_patterns_2_digit_naics_aggregate_county_historic_2020-01.csv'))
naics_feb = read_csv(str_c(fp, 'PROCESSED_DATA/historic_safegraph/safegraph_daily_patterns_2_digit_naics_aggregate_county_historic_2020-02.csv'))
fb <- read_csv(str_c(fp, 'PROCESSED_DATA/fb_mobility.csv'))

Parsed with column specification:
cols(
  county_fips = col_integer(),
  device_count = col_integer(),
  completely_home_device_count = col_integer(),
  part_time_work_behavior_devices = col_integer(),
  full_time_work_behavior_devices = col_integer(),
  ds = col_date(format = "")
)
Parsed with column specification:
cols(
  fips = col_integer(),
  name = col_character(),
  state_abbv = col_character(),
  type = col_character(),
  dt = col_date(format = "")
)
Parsed with column specification:
cols(
  county_fips = col_integer(),
  county_name = col_character(),
  state_abbv = col_character(),
  long = col_double(),
  lat = col_double()
)
Parsed with column specification:
cols(
  county_fips = col_character(),
  dt = col_date(format = ""),
  PRCP = col_double(),
  TMAX = col_double()
)
Parsed with column specification:
cols(
  county_fips = col_integer(),
  djt_vs = col_double(),
  hrc_vs = col_double(),
  djt_margin = col_double()
)
Parsed with column specification:
cols(
  NAICS = col_

In [3]:
# Clean weather data

weather %>%
    mutate(county_fips = as.numeric(county_fips)) %>%
    rename(ds = dt) -> weather

weather %>%
    inner_join(weather %>%
               group_by(county_fips) %>%
               tally() %>%
               filter(n == max(n)) %>%
               select(-n)) -> weather

Joining, by = "county_fips"


In [4]:
# Create different weights to weight data points in the model
    # However, based on our testing, weights generally worsens F stat, so we do not use weights in current models

# census weights 
census %>% filter(YEAR==11) %>% filter(AGEGRP == 0) %>% 
    mutate(county_fips=as.numeric(paste0(STATE,COUNTY))) %>% 
    select(county_fips, TOT_POP) %>% mutate(w_pop = TOT_POP/sum(TOT_POP)) -> temp_census

# device count weights
safegraph %>% group_by(county_fips) %>% 
    summarise(max_device_count = max(device_count), mean_device_count = mean(device_count)) -> n_df
n_df %>% mutate(norm_max_device = max_device_count/sum(max_device_count), 
               norm_mean_device = mean_device_count/sum(mean_device_count)) -> n_df


In [5]:
# clean up work from home index data
WFH %>% mutate(NAICS = as.character(NAICS), NAICS_TITLE = as.character(NAICS_TITLE)) -> WFH
add_wfh = data.frame(NAICS = c(31, 32, 33, 44, 45, 48, 49), 
            NAICS_TITLE = c(rep("Manufacturing", 3), "Retail Trade", "Retail Trade",
                                                                      "Transportation and Warehousing", "Transportation and Warehousing"))
add_wfh %>% left_join(WFH, by = 'NAICS_TITLE') %>% select(-NAICS.y) %>% 
    mutate(NAICS_TITLE = as.character(NAICS_TITLE)) -> add_wfh
colnames(add_wfh)[1] = "NAICS"
wfh=rbind(WFH, add_wfh)

Warning message:
“Column `NAICS_TITLE` joining factor and character vector, coercing into character vector”

In [7]:
# create local baseline - share
naics_baseline = bind_rows(naics_jan, naics_feb) %>% 
    group_by(county_fips, two_digit_naics) %>%
    summarise(visit_count = sum(raw_visit_count, na.rm=TRUE)) 
naics_baseline %>% group_by(county_fips) %>% summarise(sum_visit_count = sum(visit_count)) %>% 
    right_join(naics_baseline, by = 'county_fips') %>% 
    mutate(baseline_ratio = visit_count / sum_visit_count) %>% 
    select(county_fips, two_digit_naics, baseline_ratio) %>% filter(!is.na(county_fips)) %>% 
    spread(two_digit_naics, baseline_ratio, fill = 0)-> naics_baseline


# # I originally wanted to create another version of shift, but given the time constraint, I decided to use the other version (next)
# naics_national_daily = naics %>% group_by(date, two_digit_naics) %>%
#     summarise(visit_count = sum(raw_visit_count))
# naics_national_daily %>% group_by(date) %>% 
#     summarise(sum_visit_count = sum(visit_count)) %>%
#     right_join(naics_national_daily, by = 'date') %>% 
#     mutate(national_ratio = visit_count / sum_visit_count) %>% 
#     select(date, two_digit_naics, national_ratio) %>%
#     spread(two_digit_naics, national_ratio, fill = 0) -> naics_national_daily

# create national shock - shift: percent change relative to the baseline
naics_national_change = naics %>% group_by(date, two_digit_naics) %>%
    summarise(visit_count = sum(raw_visit_count))
naics_national_change2 = bind_rows(naics_jan, naics_feb) %>% 
    group_by(date, two_digit_naics) %>% 
    summarise(daily_visit = sum(raw_visit_count, na.rm=TRUE)) %>% 
    group_by(two_digit_naics) %>% 
    summarise(mean_visit = mean(daily_visit, na.rm=TRUE)) %>%
    right_join(naics_national_change, by = c('two_digit_naics')) %>% 
    mutate(percent_change = (visit_count - mean_visit) / mean_visit) 
naics_national_change2 %>% select(two_digit_naics, date, percent_change) %>%
    spread(two_digit_naics, percent_change, fill = 0) -> naics_national_change
for (i in 2:ncol(naics_national_change)){
    naics_code = colnames(naics_national_change)[i]
    colnames(naics_national_change)[i] = paste0('national_percent_change', naics_code)
}
head(naics_national_change)

# create national shock - shift: log change 
# In this case, we don’t actually have to take the difference from baseline because of the fixed effects
naics_national_logchange = naics %>% group_by(date, two_digit_naics) %>%
    summarise(visit_count = sum(raw_visit_count)) %>%
    mutate(log_visit = log(visit_count + 1)) 
naics_national_logchange %>% select(two_digit_naics, date, log_visit) %>%
    spread(two_digit_naics, log_visit, fill = 0) -> naics_national_logchange
for (i in 2:ncol(naics_national_logchange)){
    naics_code = colnames(naics_national_logchange)[i]
    colnames(naics_national_logchange)[i] = paste0('national_logchange', naics_code)
}
head(naics_national_logchange)

date,national_percent_change11,national_percent_change21,national_percent_change22,national_percent_change23,national_percent_change31,national_percent_change32,national_percent_change33,national_percent_change42,national_percent_change44,⋯,national_percent_change54,national_percent_change55,national_percent_change56,national_percent_change61,national_percent_change62,national_percent_change71,national_percent_change72,national_percent_change81,national_percent_change92,national_percent_change<NA>
2020-03-01,-0.1960903,-0.2866148,-0.554909863,-0.203950377,-0.16747678,-0.41577989,-0.37153705,-0.4100758608,-0.12706329,⋯,-0.357598602,-0.25236022,-0.2358388,-0.6224204,-0.43380089,-0.12126422,-0.06065782,0.3486316,-0.434545991,-0.23997555
2020-03-02,0.1556299,1.0760664,0.167156229,0.021891372,-0.17814418,0.02717939,-0.01706807,0.0785150068,-0.04689570,⋯,0.022871874,0.15817871,-0.0726113,0.2831114,0.10933164,-0.07195818,-0.14357157,-0.1884929,0.098011216,-0.20841113
2020-03-03,0.2263134,0.3027947,0.130098750,-0.017143810,-0.04297749,0.03041008,-0.04571776,0.0747500709,-0.06980854,⋯,-0.041769371,0.14096920,-0.1709014,0.2716573,0.08808970,-0.02251231,-0.12676117,-0.1692332,0.135174436,-0.22274278
2020-03-04,0.2602662,0.3521748,0.101589286,-0.093377533,-0.15506764,-0.01650518,-0.08392678,-0.0002229526,-0.12351583,⋯,-0.105934712,-0.04958018,-0.2091149,0.2095242,0.02798572,-0.08613065,-0.14179123,-0.0960859,0.002556895,-0.24829191
2020-03-05,0.2258504,0.2565665,0.122052424,-0.027941702,-0.06586137,0.06180733,-0.02328306,0.0788720347,-0.06067817,⋯,-0.047811912,0.01985318,-0.1500849,0.2711603,0.05396796,-0.01422684,-0.06711299,-0.1878916,0.049184597,-0.21467523
2020-03-06,0.3726191,1.4479933,-0.002660803,-0.001406134,0.11518849,0.04326596,-0.01214467,0.1192389835,0.03956161,⋯,0.003748218,0.14107743,-0.1201417,0.1271388,-0.03070059,0.02116551,0.13518725,-0.1616380,0.024461064,-0.06441004


date,national_logchange11,national_logchange21,national_logchange22,national_logchange23,national_logchange31,national_logchange32,national_logchange33,national_logchange42,national_logchange44,⋯,national_logchange54,national_logchange55,national_logchange56,national_logchange61,national_logchange62,national_logchange71,national_logchange72,national_logchange81,national_logchange92,national_logchange<NA>
2020-03-01,8.558335,6.522093,11.43245,12.11673,13.77397,11.37761,11.95261,13.33953,17.99016,⋯,13.41231,9.533583,13.43481,16.24271,16.70814,17.36211,18.40280,16.75233,13.96172,13.05487
2020-03-02,8.921191,7.589336,12.39649,12.36648,13.76108,11.94190,12.39987,13.94287,18.07802,⋯,13.87747,9.971240,13.62841,17.46597,17.38071,17.41670,18.31039,16.24438,14.62535,13.09556
2020-03-03,8.980550,7.123673,12.36423,12.32753,13.91334,11.94504,12.37029,13.93937,18.05369,⋯,13.81219,9.956270,13.51637,17.45700,17.36137,17.46861,18.32983,16.26784,14.65863,13.07729
2020-03-04,9.007857,7.160846,12.33868,12.24680,13.78877,11.89844,12.32942,13.86706,17.99422,⋯,13.74288,9.773550,13.46919,17.40691,17.30455,17.40131,18.31247,16.35222,14.53440,13.04386
2020-03-05,8.980172,7.087574,12.35708,12.31649,13.88914,11.97505,12.39352,13.94320,18.06345,⋯,13.80586,9.844056,13.54117,17.45661,17.32951,17.47705,18.39590,16.24512,14.57986,13.08761
2020-03-06,9.093245,7.754053,12.23926,12.34342,14.06629,11.95744,12.40486,13.97994,18.16485,⋯,13.85859,9.956365,13.57579,17.33636,17.24577,17.51232,18.59217,16.27694,14.55601,13.26269


In [8]:
### Data Preprocessing
census %>% 
  filter(YEAR == 11) %>% filter(AGEGRP==0) %>%
  group_by(STATE, COUNTY) %>% 
  summarise(n = sum(TOT_POP)) %>% 
  ungroup() %>%
  mutate(COUNTY = str_c(STATE, COUNTY)) %>%
  rename(county_fips = COUNTY) %>% 
  select(-STATE) %>% 
  mutate(county_fips = as.numeric(county_fips)) -> population

counties_long %>% 
    mutate(value = 1) %>%
    spread(key = type, value = value) -> countyPolicy

safegraph %>%
    group_by(county_fips) %>%
    summarize(n = n()) %>%
    filter(n == max(n)) %>%
    select(county_fips) %>%
    inner_join(sci %>% 
               ungroup() %>% 
               select(county_fips = user_county) %>% 
               distinct()) %>%
    inner_join(countyPolicy %>% 
               select(county_fips = fips) %>% 
               distinct()) %>%
    semi_join(weather %>% 
              select(county_fips) %>% 
              distinct()) %>%
    semi_join(fb %>%
              group_by(county_fips) %>%
              tally() %>%
              filter(n == max(n))) -> fips


safegraph %>%
    inner_join(fips) %>%
    arrange(county_fips, ds) %>%
    mutate(home_device_frac = completely_home_device_count/device_count,
           work_device_frac = (completely_home_device_count + part_time_work_behavior_devices)/device_count) %>%
    inner_join(countyInfo %>% select(-county_name)) %>%
    select(ds, county_fips, home_device_frac, work_device_frac, state_abbv) %>%
    left_join(countyPolicy, by = c('ds' = 'dt', 'county_fips' = 'fips', 'state_abbv')) %>%
    group_by(county_fips) %>%
    arrange(county_fips, ds) %>%
    fill(gatherings50, gatherings500, gyms_movies, restaurants, schools, stay_home) %>%
    replace_na(list(gatherings50 = 0, gatherings500 = 0, 
                    gyms_movies = 0, restaurants = 0, 
                    schools = 0, stay_home = 0)) %>%
    select(-name) %>%
    left_join(weather)%>%
#     left_join(naics_national_daily, by = c("ds" = "date", 'county_fips')) %>%
    # based on the most updated disucssion, we limit the analysis period to be 3/1-4/18.
    filter(ds >= as.Date('2020-03-01')) %>% 
    filter(ds <= as.Date('2020-04-18')) %>% 
    left_join(fb) %>%
    inner_join(population) %>%
    left_join(county_mobility_dvs, by = c('county_fips' = 'origin_county', 'ds')) %>% 
    left_join(naics_national_change, by = c('ds' = 'date')) %>%
    left_join(naics_baseline, by = 'county_fips') %>% 
    left_join(naics_national_logchange, by = c('ds' = 'date'))-> df

df %>%
    ungroup() %>%
    select(ds) %>%
    distinct() %>%
    arrange(ds) -> dates

head(df)
# when joining with naics visit data, there will be NA produced because there are missing data 
#         for some counties and some industries. So I set all these missing data to be 0.
# df[is.na(df)]=0

Joining, by = "county_fips"
Joining, by = "county_fips"
Joining, by = "county_fips"
Joining, by = "county_fips"
Joining, by = "county_fips"
Joining, by = "county_fips"
Joining, by = c("ds", "county_fips")
Joining, by = c("ds", "county_fips")
Joining, by = "county_fips"


ds,county_fips,home_device_frac,work_device_frac,state_abbv,gatherings50,gatherings500,gyms_movies,restaurants,schools,⋯,national_logchange54,national_logchange55,national_logchange56,national_logchange61,national_logchange62,national_logchange71,national_logchange72,national_logchange81,national_logchange92,national_logchange<NA>
2020-03-01,1001,0.2362245,0.3314626,AL,0,0,0,0,0,⋯,13.41231,9.533583,13.43481,16.24271,16.70814,17.36211,18.40280,16.75233,13.96172,13.05487
2020-03-02,1001,0.1754155,0.3324009,AL,0,0,0,0,0,⋯,13.87747,9.971240,13.62841,17.46597,17.38071,17.41670,18.31039,16.24438,14.62535,13.09556
2020-03-03,1001,0.1662855,0.3502123,AL,0,0,0,0,0,⋯,13.81219,9.956270,13.51637,17.45700,17.36137,17.46861,18.32983,16.26784,14.65863,13.07729
2020-03-04,1001,0.1840174,0.3708709,AL,0,0,0,0,0,⋯,13.74288,9.773550,13.46919,17.40691,17.30455,17.40131,18.31247,16.35222,14.53440,13.04386
2020-03-05,1001,0.1558463,0.3332229,AL,0,0,0,0,0,⋯,13.80586,9.844056,13.54117,17.45661,17.32951,17.47705,18.39590,16.24512,14.57986,13.08761
2020-03-06,1001,0.1412305,0.3336166,AL,0,0,0,0,0,⋯,13.85859,9.956365,13.57579,17.33636,17.24577,17.51232,18.59217,16.27694,14.55601,13.26269


In [9]:
df %>% 
    mutate(shiftshare1_11 = national_percent_change11 * `11`,
           shiftshare1_21 = national_percent_change21 * `21`,
           shiftshare1_22 = national_percent_change22 * `22`,
           shiftshare1_23 = national_percent_change23 * `23`,
           shiftshare1_31 = national_percent_change31 * `31`,
           shiftshare1_32 = national_percent_change32 * `32`,
           shiftshare1_33 = national_percent_change33 * `33`,
           shiftshare1_42 = national_percent_change42 * `42`,
           shiftshare1_44 = national_percent_change44 * `44`,
           shiftshare1_45 = national_percent_change45 * `45`,
           shiftshare1_48 = national_percent_change48 * `48`,
           shiftshare1_49 = national_percent_change49 * `49`,
           shiftshare1_51 = national_percent_change51 * `51`,
           shiftshare1_52 = national_percent_change52 * `52`,
           shiftshare1_53 = national_percent_change53 * `53`,
           shiftshare1_54 = national_percent_change54 * `54`,
           shiftshare1_55 = national_percent_change55 * `55`,
           shiftshare1_56 = national_percent_change56 * `56`,
           shiftshare1_61 = national_percent_change61 * `61`,
           shiftshare1_62 = national_percent_change62 * `62`,
           shiftshare1_71 = national_percent_change71 * `71`,
           shiftshare1_72 = national_percent_change72 * `72`,
           shiftshare1_81 = national_percent_change81 * `81`,
           shiftshare1_92 = national_percent_change92 * `92`,
           shiftshare1_NA = `national_percent_change<NA>` * `<NA>`,
           shiftshare2_11 = national_logchange11 * `11`,
           shiftshare2_21 = national_logchange21 * `21`,
           shiftshare2_22 = national_logchange22 * `22`,
           shiftshare2_23 = national_logchange23 * `23`,
           shiftshare2_31 = national_logchange31 * `31`,
           shiftshare2_32 = national_logchange32 * `32`,
           shiftshare2_33 = national_logchange33 * `33`,
           shiftshare2_42 = national_logchange42 * `42`,
           shiftshare2_44 = national_logchange44 * `44`,
           shiftshare2_45 = national_logchange45 * `45`,
           shiftshare2_48 = national_logchange48 * `48`,
           shiftshare2_49 = national_logchange49 * `49`,
           shiftshare2_51 = national_logchange51 * `51`,
           shiftshare2_52 = national_logchange52 * `52`,
           shiftshare2_53 = national_logchange53 * `53`,
           shiftshare2_54 = national_logchange54 * `54`,
           shiftshare2_55 = national_logchange55 * `55`,
           shiftshare2_56 = national_logchange56 * `56`,
           shiftshare2_61 = national_logchange61 * `61`,
           shiftshare2_62 = national_logchange62 * `62`,
           shiftshare2_71 = national_logchange71 * `71`,
           shiftshare2_72 = national_logchange72 * `72`,
           shiftshare2_81 = national_logchange81 * `81`,
           shiftshare2_92 = national_logchange92 * `92`,
           shiftshare2_NA = `national_logchange<NA>` * `<NA>`) -> df

head(df)

ds,county_fips,home_device_frac,work_device_frac,state_abbv,gatherings50,gatherings500,gyms_movies,restaurants,schools,⋯,shiftshare2_54,shiftshare2_55,shiftshare2_56,shiftshare2_61,shiftshare2_62,shiftshare2_71,shiftshare2_72,shiftshare2_81,shiftshare2_92,shiftshare2_NA
2020-03-01,1001,0.2362245,0.3314626,AL,0,0,0,0,0,⋯,0.05103852,0.01275576,0.005933215,1.934712,0.7012790,1.120921,4.237670,0.6399740,0.1058898,0.02086261
2020-03-02,1001,0.1754155,0.3324009,AL,0,0,0,0,0,⋯,0.05280860,0.01334133,0.006018712,2.080418,0.7295082,1.124445,4.216391,0.6205691,0.1109229,0.02092764
2020-03-03,1001,0.1662855,0.3502123,AL,0,0,0,0,0,⋯,0.05256018,0.01332130,0.005969234,2.079350,0.7286967,1.127796,4.220867,0.6214652,0.1111754,0.02089844
2020-03-04,1001,0.1840174,0.3708709,AL,0,0,0,0,0,⋯,0.05229644,0.01307683,0.005948395,2.073383,0.7263117,1.123452,4.216869,0.6246889,0.1102332,0.02084503
2020-03-05,1001,0.1558463,0.3332229,AL,0,0,0,0,0,⋯,0.05253611,0.01317116,0.005980185,2.079303,0.7273594,1.128341,4.236082,0.6205974,0.1105780,0.02091494
2020-03-06,1001,0.1412305,0.3336166,AL,0,0,0,0,0,⋯,0.05273678,0.01332143,0.005995477,2.064980,0.7238444,1.130619,4.281278,0.6218129,0.1103971,0.02119473


In [10]:
# clean up sci data

sci %>%
    inner_join(fips, by = c('user_county' = 'county_fips')) %>%
    inner_join(fips, by = c('fr_county' = 'county_fips')) %>%
    left_join(population, by = c('fr_county' = 'county_fips')) %>%
    mutate(w = ifelse(user_county == fr_county, 0 , scaled_sci * n)) %>%
    group_by(user_county) %>%
    mutate(w = w/sum(w)) %>% 
    arrange(user_county, fr_county) -> sci

head(sci)

user_county,fr_county,scaled_sci,n,w
1001,1001,10197733,55601,0.000000000
1001,1003,80006,218022,0.011608125
1001,1005,93083,24881,0.001541265
1001,1007,167754,22400,0.002500691
1001,1009,48899,57840,0.001882207
1001,1011,200281,10138,0.001351236


In [11]:
# Generating Weighting Matrices
sci %>% 
    select(user_county, fr_county, w) %>%
    spread(key = fr_county, value = w) %>%
    ungroup() %>%
    select(-user_county) %>%
    as.matrix() -> WM

# geo adj matrix, rows are origin county, columns are destination.
load('/nfs/sloanlab004/projects/covid_mobility_proj/data/PROCESSED_DATA/geo_adjacency_matrix.Rdata')
as.data.frame(geo_adj_matrix) %>%
    mutate(origin_county = colnames(geo_adj_matrix)) %>%
    gather(key = 'dest_county', value = 'w', -origin_county) %>%
    mutate(origin_county = as.numeric(origin_county),
           dest_county = as.numeric(dest_county),
           w = ifelse(origin_county == dest_county, 0 , w)) %>%
    inner_join(fips, by = c('origin_county' = 'county_fips')) %>%
    inner_join(fips, by = c('dest_county' = 'county_fips')) %>%
    group_by(dest_county) %>%
    mutate(w = w / sum(w)) %>%
    spread(key = origin_county, value = w) %>%
    ungroup() %>%
    select(-dest_county) %>%
    as.matrix() -> gWM

In [12]:
# function that constructs instruments

weightedAlters <- function(wm, colname, ...) {
    df %>% 
        select(ds, county_fips, ...) %>%
        spread(key = county_fips, value = ...) %>%
        ungroup() %>%
        select(-ds) %>%
        as.matrix() -> txn_data
    
    outMatrix <- tcrossprod(txn_data, wm)
    colnames(outMatrix) <- fips$county_fips
    
    data.frame(dates, outMatrix) %>%
        gather(key = 'county_fips', value = 'value', -ds) %>%
        mutate(county_fips = as.integer(str_sub(county_fips, 2, -1))) %>% 
        arrange(ds, county_fips) %>%
        select(-ds, -county_fips) -> out_df
    colnames(out_df)[1] <- colname
    return(out_df)
}


In [13]:
# Constructing industry share instruments - relative change

instru_index = grep('shiftshare1', colnames(df))
# get inner product term by sum up all the interaction terms
df$shiftshare1 = rowSums(df[,instru_index])
instru_index = grep('shiftshare1', colnames(df))

# get the alters as instruments
for (i in instru_index){
    # alter of industry shares
    temp_colname=colnames(df)[i]
    colname = paste0('alter_',temp_colname)
    temp = weightedAlters(WM, colname, temp_colname)
    df %>% bind_cols(temp) -> df
}
head(df)

ds,county_fips,home_device_frac,work_device_frac,state_abbv,gatherings50,gatherings500,gyms_movies,restaurants,schools,⋯,alter_shiftshare1_55,alter_shiftshare1_56,alter_shiftshare1_61,alter_shiftshare1_62,alter_shiftshare1_71,alter_shiftshare1_72,alter_shiftshare1_81,alter_shiftshare1_92,alter_shiftshare1_NA,alter_shiftshare1
2020-03-01,1001,0.2362245,0.3314626,AL,0,0,0,0,0,⋯,-2.650400e-05,-0.0003179066,-0.05332810,-0.03461545,-0.008635082,-0.01440031,0.01247329,-0.003070379,-0.0002295235,-0.1262469
2020-03-02,1001,0.1754155,0.3324009,AL,0,0,0,0,0,⋯,-2.344663e-05,-0.0004149520,-0.04994640,-0.03399690,-0.009065756,-0.01460443,0.01234269,-0.002631444,-0.0003296693,-0.1229047
2020-03-03,1001,0.1662855,0.3502123,AL,0,0,0,0,0,⋯,-3.825611e-05,-0.0003862573,-0.04695490,-0.03361983,-0.008305640,-0.01523887,0.01201560,-0.003045741,-0.0003000334,-0.1190357
2020-03-04,1001,0.1840174,0.3708709,AL,0,0,0,0,0,⋯,-3.304481e-05,-0.0003531005,-0.05243129,-0.03447113,-0.008786511,-0.01523348,0.01265627,-0.002731847,-0.0002311988,-0.1280940
2020-03-05,1001,0.1558463,0.3332229,AL,0,0,0,0,0,⋯,-2.731387e-05,-0.0004042274,-0.05052423,-0.03598563,-0.007937115,-0.01452305,0.01221593,-0.002818113,-0.0003474660,-0.1238812
2020-03-06,1001,0.1412305,0.3336166,AL,0,0,0,0,0,⋯,-1.975709e-05,-0.0003063283,-0.05133412,-0.03521334,-0.008296107,-0.01517417,0.01194276,-0.002951742,-0.0002481434,-0.1242498


In [14]:
# Constructing industry share instruments - log change

instru_index = grep('shiftshare2', colnames(df))
# get inner product term by sum up all the interaction terms
df$shiftshare2 = rowSums(df[,instru_index])
instru_index = grep('shiftshare2', colnames(df))

# get the alters as instruments
for (i in instru_index){
    # alter of industry shares
    temp_colname=colnames(df)[i]
    colname = paste0('alter_',temp_colname)
    temp = weightedAlters(WM, colname, temp_colname)
    df %>% bind_cols(temp) -> df
}
head(df)

ds,county_fips,home_device_frac,work_device_frac,state_abbv,gatherings50,gatherings500,gyms_movies,restaurants,schools,⋯,alter_shiftshare2_55,alter_shiftshare2_56,alter_shiftshare2_61,alter_shiftshare2_62,alter_shiftshare2_71,alter_shiftshare2_72,alter_shiftshare2_81,alter_shiftshare2_92,alter_shiftshare2_NA,alter_shiftshare2
2020-03-01,1001,0.2362245,0.3314626,AL,0,0,0,0,0,⋯,0.0010012594,0.01810989,1.391652,1.333238,1.236335,4.368867,0.5993626,0.09864956,0.01248627,17.53393
2020-03-02,1001,0.1754155,0.3324009,AL,0,0,0,0,0,⋯,0.0008857592,0.02363819,1.303403,1.309414,1.297997,4.430797,0.5930870,0.08454683,0.01793428,17.53770
2020-03-03,1001,0.1662855,0.3502123,AL,0,0,0,0,0,⋯,0.0014452269,0.02200356,1.225337,1.294891,1.189167,4.623278,0.5773696,0.09785796,0.01632206,17.55970
2020-03-04,1001,0.1840174,0.3708709,AL,0,0,0,0,0,⋯,0.0012483561,0.02011475,1.368249,1.327679,1.258016,4.621640,0.6081553,0.08777273,0.01257740,17.54127
2020-03-05,1001,0.1558463,0.3332229,AL,0,0,0,0,0,⋯,0.0010318546,0.02302725,1.318482,1.386011,1.136403,4.406106,0.5869958,0.09054440,0.01890244,17.54781
2020-03-06,1001,0.1412305,0.3336166,AL,0,0,0,0,0,⋯,0.0007463768,0.01745032,1.339617,1.356266,1.187802,4.603649,0.5738697,0.09483783,0.01349920,17.55356


In [19]:

# get the finalized panel data for analysis
df %>%
    ungroup() %>%
    arrange(ds, county_fips) -> panel
save(panel, file = '/nfs/sloanlab004/projects/covid_mobility_proj/data/PROCESSED_DATA/data_shiftshare.RData')

In [16]:
colnames(panel); head(panel)

[1] "ds"                                "county_fips"                      
  [3] "home_device_frac"                  "work_device_frac"                 
  [5] "state_abbv"                        "gatherings50"                     
  [7] "gatherings500"                     "gyms_movies"                      
  [9] "restaurants"                       "schools"                          
 [11] "stay_home"                         "PRCP"                             
 [13] "TMAX"                              "X1"                               
 [15] "fb_btvrc"                          "fb_rstu"                          
 [17] "n"                                 "device_count"                     
 [19] "non_home_cbg_visits_within_county" "cbg_visits_outside_county"        
 [21] "home_cbg_visits"                   "outside_device_county_visits"     
 [23] "national_percent_change11"         "national_percent_change21"        
 [25] "national_percent_change22"         "national_percent_change23"        
 [27] "national_percent_change31"         "national_percent_change32"        
 [29] "national_percent_change33"         "national_percent_change42"        
 [31] "national_percent_change44"         "national_percent_change45"        
 [33] "national_percent_change48"         "national_percent_change49"        
 [35] "national_percent_change51"         "national_percent_change52"        
 [37] "national_percent_change53"         "national_percent_change54"        
 [39] "national_percent_change55"         "national_percent_change56"        
 [41] "national_percent_change61"         "national_percent_change62"        
 [43] "national_percent_change71"         "national_percent_change72"        
 [45] "national_percent_change81"         "national_percent_change92"        
 [47] "national_percent_change<NA>"       "11"                               
 [49] "21"                                "22"                               
 [51] "23"                                "31"                               
 [53] "32"                                "33"                               
 [55] "42"                                "44"                               
 [57] "45"                                "48"                               
 [59] "49"                                "51"                               
 [61] "52"                                "53"                               
 [63] "54"                                "55"                               
 [65] "56"                                "61"                               
 [67] "62"                                "71"                               
 [69] "72"                                "81"                               
 [71] "92"                                "<NA>"                             
 [73] "national_logchange11"              "national_logchange21"             
 [75] "national_logchange22"              "national_logchange23"             
 [77] "national_logchange31"              "national_logchange32"             
 [79] "national_logchange33"              "national_logchange42"             
 [81] "national_logchange44"              "national_logchange45"             
 [83] "national_logchange48"              "national_logchange49"             
 [85] "national_logchange51"              "national_logchange52"             
 [87] "national_logchange53"              "national_logchange54"             
 [89] "national_logchange55"              "national_logchange56"             
 [91] "national_logchange61"              "national_logchange62"             
 [93] "national_logchange71"              "national_logchange72"             
 [95] "national_logchange81"              "national_logchange92"             
 [97] "national_logchange<NA>"            "shiftshare1_11"                   
 [99] "shiftshare1_21"                    "shiftshare1_22"                   
[101] "shiftshare1_23"                    "shiftshare1_31"                   
[103] "shiftshare1_32"  

ds,county_fips,home_device_frac,work_device_frac,state_abbv,gatherings50,gatherings500,gyms_movies,restaurants,schools,⋯,alter_shiftshare2_55,alter_shiftshare2_56,alter_shiftshare2_61,alter_shiftshare2_62,alter_shiftshare2_71,alter_shiftshare2_72,alter_shiftshare2_81,alter_shiftshare2_92,alter_shiftshare2_NA,alter_shiftshare2
2020-03-01,1001,0.2362245,0.3314626,AL,0,0,0,0,0,⋯,0.0010012594,0.01810989,1.391652,1.333238,1.236335,4.368867,0.5993626,0.09864956,0.01248627,17.53393
2020-03-01,1003,0.2636410,0.3450718,AL,0,0,0,0,0,⋯,0.0015667677,0.02486826,1.409765,1.308530,1.155430,4.427082,0.6070288,0.09506954,0.02298163,17.53747
2020-03-01,1005,0.2741062,0.3759480,AL,0,0,0,0,0,⋯,0.0012332884,0.02461366,1.479360,1.431615,1.086424,4.360208,0.6621100,0.09557435,0.01582506,17.51704
2020-03-01,1007,0.2444347,0.3387167,AL,0,0,0,0,0,⋯,0.0006396170,0.02142617,1.640196,1.473676,1.124210,4.199133,0.6589090,0.11219177,0.01823936,17.47589
2020-03-01,1009,0.2280778,0.3282520,AL,0,0,0,0,0,⋯,0.0001357403,0.03084086,1.159963,1.290770,1.721722,4.436004,0.5439507,0.07996345,0.02105778,17.53849
2020-03-01,1011,0.3032368,0.3798978,AL,0,0,0,0,0,⋯,0.0001647777,0.03190591,1.248809,1.358519,1.705464,4.345497,0.6150841,0.09231237,0.01486943,17.50076
